In [19]:
def whatRound(round, elimination, maxRounds):
    # if single elimination
    if elimination == "s":
        if maxRounds == 4:
            roundName = {
                4 : "Finals",
                3 : "Semi-Finals",
                2 : "Quarter Finals",
                1 : "Ro16"
            }
            return roundName[round]
        elif maxRounds == 5:
            roundName = {
                5 : "Finals",
                4 : "Semi-Finals",
                3 : "Quarter Finals",
                2 : "Ro16",
                1 : "Ro32"
            }
            return roundName[round]
        elif maxRounds == 6:
            roundName = {
                6 : "Finals",
                5 : "Semi-Finals",
                4 : "Quarter Finals",
                3 : "Ro16",
                2 : "Ro32",
                1 : "Ro64"
            }
            return roundName[round]
        else:
            print(f"There is no name for a tournament with a max round of {maxRounds}")
            return
    # if double elimination
    elif elimination == "d":
        if maxRounds == 4:
            roundName = {
                4 : "Finals",
                3 : "Winners Final",
                2 : "Winners Semi-Finals",
                1 : "Winners Quarter Finals",
                -1 : "Losers Round 1",
                -2 : "Losers Quarter Finals",
                -3 : "Losers Semi-Finals",
                -4 : "Losers Finals"
            }
            return roundName[round]
        elif maxRounds == 5:
            roundName = {
                5 : "Finals",
                4 : "Winners Final",
                3 : "Winners Semi-Finals",
                2 : "Winners Quarter Finals",
                1 : "Winners Ro16",
                -1 : "Losers Round 1",
                -2 : "Losers Round 2",
                -3 : "Losers Quarter Finals",
                -4 : "Losers Semi-Finals",
                -5 : "Losers Final"
            }
            return roundName[round]
        else:
            print(f"There is no name for a tournament with a max round of {maxRounds}")
            return
    else:
        print("No elimination type given")
        return


In [37]:
import sqlite3
def numberOfMMC(NAME):
    conn: sqlite3.Connection = sqlite3.connect("mmc.db")
    conn.execute("PRAGMA foreign_keys = 1")
    c: sqlite3.Cursor = conn.cursor()

    cnames = [] # for collecting all the challonge names a player has had
    specificMMC = [] # for recording the specific editions a player has played in
    c.execute("SELECT * FROM ChallongeNames WHERE NAME = ?", (NAME,)) # query all challonge names a player has had
    results = c.fetchall()
    for r in results:
        cnames.append(r[0])
    for cn in cnames:
        c.execute("SELECT * FROM Participants WHERE CNAME = ?",(cn,))
        results = c.fetchall()
        for r in results:
            c.execute("SELECT NUMBER FROM MMC WHERE TOURNAMENTID = ?", (r[3],))
            results2 = c.fetchone()
            if results2[0]:
                specificMMC.append(results2[0])
    specificMMC.sort()
    # print results
    if len(specificMMC) == 1:
        print(f"{NAME} has played in {len(specificMMC)} MMC")
        print(f"The specific edition {NAME} has played in is: {specificMMC[0]}")
    else:
        print(f"{NAME} has played in {len(specificMMC)} MMC's")
        print(f"The specific editions {NAME} has played in are:")
        num = 0
        for i in range(len(specificMMC) - 1):
            if num < 10:
                print(f"{specificMMC[i]}, ", end="")
                num += 1
            else:
                print(f"{specificMMC[i]}")
                num = 0
        print(specificMMC[-1])
    conn.close()
    return

In [21]:
import sqlite3
def winLossRecord(NAME):
    conn: sqlite3.Connection = sqlite3.connect("mmc.db")
    conn.execute("PRAGMA foreign_keys = 1")
    c: sqlite3.Cursor = conn.cursor()

    cnames = [] # for collecting all the challonge names a player has had
    cid = set() # for saving all of the challonge id's
    matchesWon = [] # for matches won 
    matchesLost = [] # for matches lost
    maps = {"maps won": 0, "maps lost": 0} # for tracking maps won and lost
    scores = {"1-0": 0, "0-1": 0, "2-0": 0, "0-2": 0, "2-1": 0, "1-2": 0, "3-0": 0, "3-1": 0, "3-2": 0, "0-3": 0, "1-3": 0, "2-3": 0}
    woWon = [] # for matches won by walkover
    woLost = [] # for matches lost by walkover

    # get the challonge names used on the brackets
    c.execute("SELECT * FROM ChallongeNames WHERE NAME = ?", (NAME,)) # query all challonge names a player has had
    results = c.fetchall()
    for r in results:
        cnames.append(r[0])

    # get the challonge id's that will appear in the matches
    for cn in cnames:
        c.execute("SELECT * FROM Participants WHERE CNAME = ?",(cn,))
        results = c.fetchall()
        for r in results:
            cid.add(r[0])

    # get all the matches a player has played
    for ids in cid:
        # get all the wins
        c.execute("SELECT * FROM MATCHES WHERE WINNERID = ?", (ids,))
        results = c.fetchall()
        for r in results:
            # check if it was a walkover
            if r[5] == -1:
                woWon.append(r)
            else:
                maps["maps won"] += r[4]
                maps["maps lost"] += r[5]
                scores[f"{r[4]}-{r[5]}"] += 1
                matchesWon.append(r)
        # get all thheh losses
        c.execute("SELECT * FROM MATCHES WHERE LOSERID = ?", (ids,))
        results = c.fetchall()
        for r in results:
            # check if it was a walkover
            if r[5] == -1:
                woLost.append(r)
            else:
                maps["maps won"] += r[5]
                maps["maps lost"] += r[4]
                scores[f"{r[5]}-{r[4]}"] += 1
                matchesLost.append(r)

    # display results
    print(f"{NAME} Career Record: {len(matchesWon)+len(woWon)}-{len(matchesLost)+len(woLost)}  Maps: {maps['maps won']}-{maps['maps lost']}")
    print(f"Wins: {len(matchesWon)}")
    print(f"Losses: {len(matchesLost)}")
    print(f"Wins by Walkover: {len(woWon)}")
    print(f"Losses by Walkover: {len(woLost)}")
    print("Map Scores:")
    num = 0
    for key, value in scores.items():
        if value > 0:
            if num < 5:
                print(f"{key}: {value}  ", end="")
                num += 1
            else:
                print(f"{key}: {value}  ")
                num = 0

    conn.close()

In [22]:
import sqlite3
def matchHistory(NAME1, NAME2):
    conn: sqlite3.Connection = sqlite3.connect("mmc.db")
    conn.execute("PRAGMA foreign_keys = 1")
    c: sqlite3.Cursor = conn.cursor()

    cnames1 = [] # for collecting all the challonge names NAME1 has had
    cnames2 = [] # for collecting all the challonge names NAME2 has had
    cid1 = set() # for saving all of the challonge id's for NAME1
    cid2 = set() # for saving all of the challonge id's for NAME2
    n1Wins = []
    n2Wins = []

    # get the challonge names used on the brackets for NAME1
    c.execute("SELECT * FROM ChallongeNames WHERE NAME = ?", (NAME1,)) # query all challonge names NAME1 has had
    results = c.fetchall()
    for r in results:
        cnames1.append(r[0])

    # get the challonge names used on the brackets for NAME2
    c.execute("SELECT * FROM ChallongeNames WHERE NAME = ?", (NAME2,)) # query all challonge names NAME2 has had
    results = c.fetchall()
    for r in results:
        cnames2.append(r[0])

    # get the challonge id's for NAME1
    for c1 in cnames1:
        c.execute("SELECT * FROM Participants WHERE CNAME = ?",(c1,))
        results = c.fetchall()
        for r in results:
            cid1.add(r[0])
    
    # get the challonge id's for NAME2
    for c2 in cnames2:
        c.execute("SELECT * FROM Participants WHERE CNAME = ?",(c2,))
        results = c.fetchall()
        for r in results:
            cid2.add(r[0])

    # check all combos of cids
    for c1 in cid1:
        for c2 in cid2:
            # wins for NAME1
            c.execute("SELECT * FROM Matches WHERE WINNERID = ? AND LOSERID = ?", (c1,c2))
            results = c.fetchall()
            for r in results:
                n1Wins.append(r)
            # wins for NAME2
            c.execute("SELECT * FROM Matches WHERE WINNERID = ? AND LOSERID = ?", (c2,c1))
            results = c.fetchall()
            for r in results:
                n2Wins.append(r)
    
    # if there are no matches between two players saw so
    if len(n1Wins) == 0 and len(n2Wins) == 0:
        print(f"{NAME1} and {NAME2} have never played each other in MMC")

    # Display Matches
    for match in n1Wins:
        c.execute("SELECT NUMBER,ELIMINATION,ROUNDS FROM MMC WHERE TOURNAMENTID = ?", (match[1],))
        mmc = c.fetchone()
        rnd = whatRound(match[6], mmc[1], mmc[2])
        print(f"{NAME1} {match[4]} - {match[5]} {NAME2}  MMC #{mmc[0]} {rnd}")
    for match in n2Wins:
        c.execute("SELECT NUMBER,ELIMINATION,ROUNDS FROM MMC WHERE TOURNAMENTID = ?", (match[1],))
        mmc = c.fetchone()
        rnd = whatRound(match[6], mmc[1], mmc[2])
        print(f"{NAME2} {match[4]} - {match[5]} {NAME1}  MMC #{mmc[0]} {rnd}")

### How to see how many MMC's a player has played in

In [42]:
# Who do you want to see the number of MMC's of?
NAME = "CaptainClub"
numberOfMMC(NAME)


CaptainClub has played in 32 MMC's
The specific editions CaptainClub has played in are:
34, 38, 39, 40, 41, 44, 45, 46, 47, 49, 50
53, 55, 58, 61, 62, 63, 64, 65, 66, 67, 68
69, 71, 73, 74, 75, 76, 93, 97, 101, 103


### How to see someones Win/Loss Record

In [41]:
# Who do you want to see the W/L record of?
NAME = "CaptainClub"
winLossRecord(NAME)

CaptainClub Career Record: 51-38  Maps: 97-88
Wins: 44
Losses: 37
Wins by Walkover: 7
Losses by Walkover: 1
Map Scores:
1-0: 9  0-1: 6  2-0: 21  0-2: 14  2-1: 12  1-2: 12  
3-1: 1  3-2: 1  0-3: 2  1-3: 2  2-3: 1  

### How to see two players Match History

In [40]:
# Who are the two players you want to see the match history of?
NAME1 = "Patches"
NAME2 = "CaptainClub"
matchHistory(NAME1,NAME2)

Patches 2 - 0 CaptainClub  MMC #93 Winners Quarter Finals
Patches 2 - 1 CaptainClub  MMC #97 Winners Final
CaptainClub 2 - 0 Patches  MMC #63 Quarter Finals
CaptainClub 3 - 1 Patches  MMC #103 Finals
CaptainClub 2 - 0 Patches  MMC #74 Ro16


### GUI for easy viewing

In [46]:
import PySimpleGUI as sg
import pandas as pd

csvFile = "Names.csv"
df = pd.read_csv(csvFile, encoding="utf-16", delimiter="\t")

# Extract the specific column (e.g., 'Name')
names = df["Normal Name"].drop_duplicates().tolist()

# Define the layout of your PySimpleGUI window
layout = [
    [sg.Text('List of Names:')],
    [sg.Input(key='-SEARCH-', enable_events=True, size=(20, 1)), sg.Button('Search')],
    [sg.Listbox(values=names, size=(30, 10), key='-LIST-', enable_events=True)],
    [sg.Column(layout=[[sg.Text('Enter name to see MMCs:')], [sg.InputText(key='-INPUT-')], [sg.Button('Show MMCs')],
                       [sg.Button('Win/Loss Record')]], vertical_alignment='top'),
     sg.Multiline(size=(40, 10), key='-OUTPUT-', disabled=True)],
    [sg.Text('Match History:', pad=((10, 0), (10, 0))), sg.InputText(key='-INPUT-NAME1-', size=(20, 1)),
     sg.Text('vs', pad=((10, 0), (10, 0))), sg.InputText(key='-INPUT-NAME2-', size=(20, 1)), sg.Button('Match History')],
    [sg.Multiline(size=(80, 10), key='-OUTPUT-MATCHHISTORY-', disabled=True)],
    [sg.Button('Exit')]
]

# Create the PySimpleGUI window
window = sg.Window('MMC Viewer', layout)

# Event loop
while True:
    event, values = window.read()
    if event == sg.WINDOW_CLOSED or event == 'Exit':
        break
    elif event == '-LIST-':  # Handle listbox selection event
        selected_name = values['-LIST-'][0]  # Get the selected name from the listbox
        window['-INPUT-'].update(selected_name)  # Update the input field with the selected name
        output = numberOfMMC(selected_name)  # Calculate MMCs for the selected name
        window['-OUTPUT-'].update(output)  # Update the output field with the result
    elif event == 'Show MMCs':
        name = values['-INPUT-'].strip()
        if name:
            output = numberOfMMC(name)
            window['-OUTPUT-'].update(output)
    elif event == 'Search' or event == '-SEARCH-':
        search_term = values['-SEARCH-'].strip().lower()  # Get the search term and normalize to lowercase
        filtered_names = [name for name in names if search_term in name.lower()]
        window['-LIST-'].update(values=filtered_names)
    elif event == 'Win/Loss Record':
        name = values['-INPUT-'].strip()
        if name:
            output = winLossRecord(name)
            window['-OUTPUT-'].update(output)
    elif event == 'Match History':
        name1 = values['-INPUT-NAME1-'].strip()
        name2 = values['-INPUT-NAME2-'].strip()
        if name1 and name2:
            output = matchHistory(name1, name2)
            window['-OUTPUT-MATCHHISTORY-'].update(output)

window.close()

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\kynan\AppData\Roaming\Python\Python310\site-packages\IPython\core\interactiveshell.py", line 3508, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\kynan\AppData\Local\Temp\ipykernel_2740\2823194460.py", line 29, in <module>
    event, values = window.read()
  File "c:\Users\kynan\AppData\Local\Programs\Python\Python310\lib\site-packages\PySimpleGUI\PySimpleGUI.py", line 10446, in read
    results = self._read(timeout=timeout, timeout_key=timeout_key)
  File "c:\Users\kynan\AppData\Local\Programs\Python\Python310\lib\site-packages\PySimpleGUI\PySimpleGUI.py", line 10526, in _read
    self._Show()
  File "c:\Users\kynan\AppData\Local\Programs\Python\Python310\lib\site-packages\PySimpleGUI\PySimpleGUI.py", line 10256, in _Show
    StartupTK(self)
  File "c:\Users\kynan\AppData\Local\Programs\Python\Python310\lib\site-packages\PySimpleGUI\PySimpleGUI.py", line 17526, in StartupTK
    __Mez2iw86(window)
 

In [ ]:
import sqlite3
conn: sqlite3.Connection = sqlite3.connect("mmc.db")
conn.execute("PRAGMA foreign_keys = 1")
c: sqlite3.Cursor = conn.cursor()

rounds = set()
c.execute("SELECT NUMBER, ROUNDS FROM MMC WHERE ELIMINATION = 's'")
results = c.fetchall()
for r in results:
    print(r)
    rounds.add(r[1])

print(max(rounds))
print(min(rounds))
# Who do you want to see the number of MMC's of?
NAME = "CaptCanuck"
cnames = [] # for collection all the challonge names a player has had
specificMMC = [] # for recording the specific editions a player has played in
cid = set() # for saving all of the challonge id's
matchesWon = []
c.execute("SELECT * FROM ChallongeNames WHERE NAME = ?", (NAME,)) # query all challonge names a player has had
results = c.fetchall()
for r in results:
    cnames.append(r[0])
for cn in cnames:
    c.execute("SELECT * FROM Participants WHERE CNAME = ?",(cn,))
    results = c.fetchall()
    print(results)
    for r in results:
        cid.add(r[0])

for id in cid:
    c.execute("SELECT * FROM MATCHES WHERE WINNERID = ?", (id,))
    results = c.fetchall()
    for r in results:
        matchesWon.append(r)

print(len(matchesWon))

conn.close()